In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import flatten_orbit as fo
import datetime as dt

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

ModuleNotFoundError: ignored

In [2]:
def datetime_to_float(d):
    # total_seconds will be in decimals (millisecond precision)
    return d.timestamp()

def float_to_datetime(fl):
    return dt.datetime.fromtimestamp(fl)

def datetime_to_timestamp_array(times):
    timestamps = np.array([time.timestamp() for time in times])
    return timestamps

def scale(X,upper = 1, lower = 0):
    ## TODO: vectorize, taking too long
    n = len(X)
    X_std = (np.array([((X[i] - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))) for i in range(0,n)]))*(upper-lower)
    X_std = X_std + lower
    return X_std

In [3]:
parse_dates = ['Time']
jupiter = pd.read_csv("Jupiter2x.csv", parse_dates=parse_dates, date_parser = pd.to_datetime)
datetimes = jupiter['Time']
timestamps = datetime_to_timestamp_array(datetimes)
jupiter_true = jupiter[['X','Y','Z']].to_numpy()
normal = fo.generate_plane_vector(jupiter_true)
jupiter_flat = fo.align_with_z_axis(jupiter_true, normal)

timestamps = timestamps - 814665600.0

FileNotFoundError: ignored

In [ ]:
def epi_x(t, params):
    """
    params is an array with the five different epicycloid parameters
    """
    (a,b,c,d,R) = params
    return (a+b)*np.cos(c*t-d) - R*np.cos((a+b)/b*(c*t-d))

In [ ]:
x_true = jupiter_flat[:,0]
x_scl = scale(x_true, upper = 1.0, lower=-1.0)
plt.plot(timestamps, epi_x(timestamps, [0.7,0.1,2e-8,0,0.2]))
plt.plot(timestamps, x_scl)
plt.show()

In [ ]:
def cost_function(x_true, timestamps, params):
    x_pred = epi_x(timestamps, params)
    cost = 0
    n = len(x_pred)
    for i in range(len(x_true)):
        cost += (x_true[i] - x_pred[i])**2
    cost = cost/(2*n)
    return cost

def calc_gradients(x_true, timestamps, params):
    x_pred = epi_x(timestamps, params)
    sin = np.sin
    cos = np.cos
    (a,b,c,d,R) = params
    
    grads = np.zeros(5)
    n = len(x_true)
    for i in range(len(x_true)):
        
        t = timestamps[i]
        delta_x = x_pred[i]-x_true[i]
        """
        Current observations
        -high timestamps create really high gradients for dedc
        -c needs to be a low number on the order of e-8. 
        -However, if c is a number this low, it seems to push down the other gradients
        -dedR kind of doesn't make sense because it's derivative is based only around a cosine
        """
        deda = R*(c*t-d)*sin((a+b)*(c*t-d)/b)/b + cos(c*t-d)
        dedb = cos(c*t-d)-a*R*(c*t-d)*sin((a+b)*(c*t-d)/b)/(b**2)
        dedc = t*(a+b)*(R*sin((a+b)*(c*t-d)/b)-b*sin(c*t-d))/b
        dedc = 0
        dedd = -(a+b)*(b*sin(d-c*t)+R*sin((a+b)*(c*t-d)/b))/b
        dedR = -cos((a+b)*(c*t-d)/b)
        
        grads[0] += delta_x*deda
        grads[1] += delta_x*dedb
        grads[2] += delta_x*dedc
        grads[3] += delta_x*dedd
        grads[4] += delta_x*dedR
    grads = grads/n
    #print("grads: ", grads)
    return grads

def update_parameters(params, gradients, learning_rate):
    for i in range(len(params)):
        params[i] = params[i] - gradients[i]*learning_rate;
        
def epi_grad_descent(x_true, timestamps, params = [0.7,0.1,1,0,0.2], iterations = 30, learning_rate=0.000001):
    costs = []
    for i in range(iterations):
        gradients = calc_gradients(x_true, timestamps, params)
        update_parameters(params, gradients, learning_rate)
        cost = cost_function(x_true, timestamps, params)
        #print("params: ", params)
        costs.append(cost)
        #print(cost)
    return params, costs

In [ ]:
params, losses = epi_grad_descent(x_scl, timestamps)
print(params)
x_pred = epi_x(timestamps, params)
plt.plot(losses)
plt.show()

In [ ]:
plt.plot(timestamps, x_pred)
plt.plot(timestamps, x_scl)
plt.show()